In [59]:
# importing the libraries

import sys
sys.path.append("..")
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D  
from rbf_layer import RBFLayer, l_norm, rbf_gaussian

In [60]:
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

M_PI=3.1416
def compute_roll_yaw_pitch(x,y,z):
  #Acceleration around X
  acc_x_accl=[]

  #Acceleration around Y
  acc_y_accl=[]

  #Acceleration arounf Z
  acc_z_accl=[]


  for (x_mean,y_mean,z_mean) in zip(x,y,z):
    acc_x_accl.append(float("{:.2f}".format(x_mean*3.9)))
    acc_y_accl.append(float("{:.2f}".format(y_mean*3.9)))
    acc_z_accl.append(float("{:.2f}".format(z_mean*3.9)))


  acc_pitch=[]
  acc_roll=[]
  acc_yaw=[]

  for (acc_x,acc_y,acc_z) in zip(acc_x_accl,acc_y_accl,acc_z_accl):
    if acc_y==0 and acc_z==0:
      value_pitch=-0.1
    else:
      value_pitch=180 * math.atan (acc_x/math.sqrt(acc_y*acc_y + acc_z*acc_z))/M_PI
    if acc_x ==0 and acc_z==0:
      value_roll=-0.1
      value_yaw=-0.1
    else:
      value_roll = 180 * math.atan (acc_y/math.sqrt(acc_x*acc_x + acc_z*acc_z))/M_PI
      value_yaw = 180 * math.atan (acc_z/math.sqrt(acc_x*acc_x + acc_z*acc_z))/M_PI
    value_pitch=float("{:.2f}".format(value_pitch))
    value_roll=float("{:.2f}".format(value_roll))
    value_yaw=float("{:.2f}".format(value_yaw))
    acc_pitch.append(value_pitch)
    acc_roll.append(value_roll)
    acc_yaw.append(value_yaw)
  return acc_pitch,acc_roll,acc_yaw

# -----------------------------

#Sliding Window to null values
def fill_null(data):
  for col in data.columns:
    null_indexes=data[data[col].isnull()].index.tolist()
    #print("For ",col)
    for ind in null_indexes:
      #print(" Got null value at ",ind)
      values=data.loc[ind-6:ind-1,col]
      #print(" Last 5 values ",values)
      mean_val=values.mean()
      data.loc[ind,col]=mean_val
  return data

# -----------------------------

# loading the data

def load_data():
    user_one = "/Users/carbon/Desktop/Federated Learning/extrasensory_dataset/00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz"
    # user_one="C:\\pdata\\extrasensory_dataset\\00EABED2-271D-49D8-B599-1D4A09240601.features_labels.csv.gz"
    df_one=pd.read_csv(user_one)
    #accelerometer
    df_acc=df_one.iloc[:,1:27]
    df_acc=fill_null(df_acc)
    #gyroscope
    df_gyro=df_one.iloc[:,27:53]
    df_gyro=fill_null(df_gyro)
    #magnometer
    df_magnet=df_one.iloc[:,53:84]
    df_magnet=fill_null(df_magnet)
    # watch accelerometer
    #df_watch_acc=df_one.iloc[:,84:130]
    # location
    #df_location=df_one.iloc[:,139:156]

    # For accelerometer
    #mean values
    acc_mean_x=df_acc['raw_acc:3d:mean_x']
    acc_mean_y=df_acc['raw_acc:3d:mean_y']
    acc_mean_z=df_acc['raw_acc:3d:mean_z']

    acc_mean_x=acc_mean_x.replace({0:0.001})

    #standard deviations
    #acc_std_x=df_acc['raw_acc:3d:std_x']
    #acc_std_y=df_acc['raw_acc:3d:std_y']
    #acc_std_z=df_acc['raw_acc:3d:std_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(acc_mean_x,acc_mean_y,acc_mean_z)
    df_one['acc_pitch']=pitch
    df_one['acc_roll']=roll
    df_one['acc_yaw']=yaw

    #for gyroscope
    gyro_mean_x=df_gyro['proc_gyro:3d:mean_x']
    gyro_mean_y=df_gyro['proc_gyro:3d:mean_y']
    gyro_mean_z=df_gyro['proc_gyro:3d:mean_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(gyro_mean_x,gyro_mean_y,gyro_mean_z)

    df_one['gyro_pitch']=pitch
    df_one['gyro_roll']=roll
    df_one['gyro_yaw']=yaw

    # For magnetometer
    magno_mean_x=df_magnet['raw_magnet:3d:mean_x']
    magno_mean_y=df_magnet['raw_magnet:3d:mean_y']
    magno_mean_z=df_magnet['raw_magnet:3d:mean_z']

    (pitch,roll,yaw)=compute_roll_yaw_pitch(magno_mean_x,magno_mean_y,magno_mean_z)

    df_one['magno_pitch']=pitch
    df_one['magno_roll']=roll
    df_one['magno_yaw']=yaw

    y=df_one[['label:FIX_running','label:FIX_walking','label:SITTING','label:SLEEPING','label:OR_standing']]

    # to avoid null values
    y['label:FIX_running']=y['label:FIX_running'].fillna(0)
    y['label:FIX_walking']=y['label:FIX_walking'].fillna(0)
    y['label:SITTING']=y['label:SITTING'].fillna(0)
    y['label:SLEEPING']=y['label:SLEEPING'].fillna(0)
    y['label:OR_standing']=y['label:OR_standing'].fillna(0)

    #Check rows where all the recorded activities are zero ~ No activity recorded rows
    list_of_indexs=[]
    for i in range(len(y)):
        run=y.iloc[i,0]
        walk=y.iloc[i,1]
        sit=y.iloc[i,2]
        sleep=y.iloc[i,3]
        stand=y.iloc[i,4]
        activities=[run,walk,sit,sleep,stand]
        count_ones=activities.count(1)
        if walk==0 and run==0 and sit==0 and sleep==0 and stand==0:
            list_of_indexs.append(i)
        #check if more then 1 exists for different activities
        elif count_ones>1:
            list_of_indexs.append(i)

    y=y.drop(list_of_indexs)
    X=df_one.iloc[:,-9:]
    X=X.drop(list_of_indexs)


    return X,y

# -----------------------------

# loading the data
X,y=load_data()
X = X.to_numpy()
labels = y.to_numpy()

# -----------------------------

# # Creating a small balanced dataset

X_1 = X[X['y'] ==  1].sample(n=150, random_state=42)
X_2 = X[X['y'] ==  2].sample(n=150, random_state=42)
X_3 = X[X['y'] ==  3].sample(n=150, random_state=42)
X_4 = X[X['y'] ==  4].sample(n=150, random_state=42)

small_X = pd.concat([X_1, X_2, X_3, X_4])
small_X = small_X.sample(frac=1).reset_index(drop=True)
small_y = small_X['y']
small_X = small_X.drop(['y'], axis=1)

In [57]:
# # Generate dataset
# N_training = 100
# N_validation = 50
# N = N_training + N_validation


# X0 = np.array([-1, 0]).T + 0.5 * np.random.normal(size=(N, 2))
# X1 = np.array([1, 0]).T +  0.5 * np.random.normal(size=(N, 2))
# X2 = np.array([0, 1]).T + 0.3 * np.random.normal(size=(N, 2))
# X = np.concatenate([X0, X1, X2])

# labels = np.zeros((3*N, 1))
# labels[N:2*N] = 1
# labels[2*N:] = 2

## creating Training Dataset

In [76]:
N = 100
N_training = int(0.8 * N)
N_validation = N - N_training

# Training dataset
indices = np.random.permutation(3 * N)
trn_indices, val_indices = indices[:N_training], indices[N_training:]
Tdataset = ([torch.tensor(X[trn_indices], dtype=torch.float32), torch.tensor(labels[trn_indices], dtype=torch.float32)])
Vdataset = ([torch.tensor(X[val_indices], dtype=torch.float32), torch.tensor(labels[val_indices], dtype=torch.float32)])

In [78]:
X.shape

(1939, 9)

In [79]:
# Define an RBF layer where the dimensionality of the input feature is 2,
# the number of kernels is 3, and 2 output features

# euclidean norm
def euclidean_norm(x):
    return torch.norm(x, p=2, dim=-1)


# Gaussian RBF
def rbf_gaussian(x):
    return (-x.pow(2)).exp()

rbf = RBFLayer(in_features_dim=9,
               num_kernels=4,
               out_features_dim=4,
               radial_function=rbf_gaussian,
               norm_function=euclidean_norm,
               normalization=True)


In [80]:
optimiser = torch.optim.Adam(rbf.parameters(), lr=1e-3)
epoch = 0
batch_size = 32
trn_losses = []
val_losses = []

for epoch in range(512):
    indices = np.random.permutation(N_training)
    batch_idx = 0
    epoch_trn_losses = []
    epoch_val_losses = []
    epoch_trn_accs = []
    epoch_val_accs = []
    
    # Epoch training
    while batch_idx < N_training:
        idxs = indices[batch_idx:batch_idx + batch_size]
        x = Tdataset[0][idxs]
        labels = Tdataset[1][idxs].flatten()
        
        # Compute loss
        optimiser.zero_grad()
        y = rbf(x)
        loss = nn.CrossEntropyLoss()(y, labels.long())
        epoch_trn_losses.append(loss.item())
        loss.backward()
        optimiser.step()
        batch_idx += batch_size
        
        with torch.no_grad():
            # Compute validation
            x = Vdataset[0]
            labels = Vdataset[1].flatten()
            y = rbf(x)
            loss = nn.CrossEntropyLoss()(y, labels.long())
            epoch_val_losses.append(loss.item())
            
    trn_losses.append(np.mean(epoch_trn_losses))
    val_losses.append(np.mean(epoch_val_losses))

plt.plot(trn_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.grid()
plt.title('Training plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

ValueError: Expected input batch_size (32) to match target batch_size (160).